# Bidirectional Recurrent Neural Networks 

In [63]:
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
import libraries.project_tests as tests
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Embedding, Activation, RepeatVector, Bidirectional, Concatenate, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.preprocessing.text import one_hot
from keras.utils import np_utils

In some problems the information required to make a prediction in one point of a sequence, includes not only pass information but also "future" information, i.e., information before and after of the target point in the sequence. This also implies that such informaction must be available to make the perdictions. For instance, in translation problems usually you need to know an entire sentence beforhand in order to translate it correctly.   The bidirectional RNNs are a modification of the standard RNNs that incorporate additional layers which transmit the information from the time $t+1$ to the time $t$. The forward and backward layers do not have any conextion among them. 

![alt text](./Images/RNN_arc_3.png "Neuronas")

## Neural Machine Translation

This example is based on the Machine Translation material included in the Deep Learning Specilization offered by Coursera: https://es.coursera.org/specializations/deep-learning

The following model architecture could be used for a full language translation problem, however it would require hundred of thousands of texts, a big computational power (GPU) and hundreds of hours in order to get a fairly accurate model. Therefore, we are going to use a medium sizes datase that includes 137860 sentences in English and French.

In [2]:
english_sentences = tuple(open('data/small_vocab_en', 'r'))
french_sentences = tuple(open('data/small_vocab_fr', 'r'))

In [3]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .

small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .

small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .



In [4]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


### Tokenize

In [5]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding

In [6]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


Applying padding to the dataset

In [7]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 200
French vocabulary size: 345


In order to see the actual French translation, it is necessary to define a function to decode the network's output.

In [8]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [9]:
def logits_to_text_en(seq, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[item] for item in seq])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [10]:
preproc_french_sentences.shape

(137860, 21, 1)

### Simple RNN network

Pay attention to the output format and the loss function.

In [22]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = 'sparse_categorical_crossentropy', 
                 optimizer = Adam(learning_rate))
    
    return model
#Input and output sequences must have the same length
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print('Original sentence:')
print(logits_to_text_en(tmp_x[:1].flatten(), english_tokenizer))
print('Translated sentence:')
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 38s 346us/step - loss: 3.4585 - val_loss: 2.5706
Epoch 2/10
110288/110288 [==============================] - 38s 340us/step - loss: 2.4560 - val_loss: 2.3349
Epoch 3/10
110288/110288 [==============================] - 38s 341us/step - loss: 2.2076 - val_loss: 2.0730
Epoch 4/10
110288/110288 [==============================] - 38s 343us/step - loss: 1.9651 - val_loss: 1.8649
Epoch 5/10
110288/110288 [==============================] - 38s 344us/step - loss: 1.7908 - val_loss: 1.7219
Epoch 6/10
110288/110288 [==============================] - 38s 343us/step - loss: 1.6743 - val_loss: 1.6305
Epoch 7/10
110288/110288 [==============================] - 38s 342us/step - loss: 1.5973 - val_loss: 1.5651
Epoch 8/10
110288/110288 [==============================] - 38s 347us/step - loss: 1.5392 - val_loss: 1.5137
Epoch 9/10
110288/110288 [==============================] - 39s 354us/step - 

In [14]:
simple_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 64)            12672     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           22425     
_________________________________________________________________
activation_4 (Activation)    (None, 21, 345)           0         
Total params: 35,097
Trainable params: 35,097
Non-trainable params: 0
_________________________________________________________________


## Using word2vec

In [29]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="relu")
    
    embedding = Embedding(english_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate))
    
    return model
#Input and output sequences must have the same length
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print('Original sentence:')
print(logits_to_text_en(tmp_x[:1].flatten(), english_tokenizer))
print('Translated sentence:')
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 31s 277us/step - loss: 3.8471 - val_loss: 2.7391
Epoch 2/10
110288/110288 [==============================] - 28s 255us/step - loss: 2.4671 - val_loss: 2.1156
Epoch 3/10
110288/110288 [==============================] - 29s 264us/step - loss: 1.7596 - val_loss: 1.4836
Epoch 4/10
110288/110288 [==============================] - 29s 263us/step - loss: 1.3190 - val_loss: 1.1665
Epoch 5/10
110288/110288 [==============================] - 32s 289us/step - loss: 1.0585 - val_loss: 0.9588
Epoch 6/10
110288/110288 [==============================] - 30s 268us/step - loss: 0.8864 - val_loss: 0.8428
Epoch 7/10
110288/110288 [==============================] - 31s 280us/step - loss: 0.7931 - val_loss: 0.7416
Epoch 8/10
110288/110288 [==============================] - 30s 270us/step - loss: 0.7083 - val_loss: 0.6713
Epoch 9/10
110288/110288 [==============================] - 33s 302us/step - 

## Bidirectional RNN

In [30]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
   
    learning_rate = 1e-3
    model = Sequential()
    model.add(Embedding(english_vocab_size, 64, input_length=input_shape[1])) 
    model.add(Bidirectional(GRU(64, return_sequences = True, dropout = 0.1)))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate))
    return model

#Input and output sequences must have the same length
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

bidi_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
# Print prediction(s)
print('Original sentence:')
print(logits_to_text_en(tmp_x[:1].flatten(), english_tokenizer))
print('Translated sentence:')
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 44s 401us/step - loss: 3.4525 - val_loss: 2.5238
Epoch 2/10
110288/110288 [==============================] - 44s 396us/step - loss: 2.2618 - val_loss: 1.8411
Epoch 3/10
110288/110288 [==============================] - 42s 383us/step - loss: 1.5661 - val_loss: 1.3373
Epoch 4/10
110288/110288 [==============================] - 52s 472us/step - loss: 1.1882 - val_loss: 1.0314
Epoch 5/10
110288/110288 [==============================] - 52s 475us/step - loss: 0.9462 - val_loss: 0.8452
Epoch 6/10
110288/110288 [==============================] - 51s 465us/step - loss: 0.7941 - val_loss: 0.7139
Epoch 7/10
110288/110288 [==============================] - 52s 468us/step - loss: 0.6853 - val_loss: 0.6191
Epoch 8/10
110288/110288 [==============================] - 50s 452us/step - loss: 0.6029 - val_loss: 0.5483
Epoch 9/10
110288/110288 [==============================] - 48s 436us/step - 

## Encoder-decoder

![alt text](./Images/EDA.png "Encoder-Decoder")

Image taken from: https://towardsdatascience.com/light-on-math-ml-attention-with-keras-dc8dbc1fad39

In [47]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 0.005
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size, output_dim=64, input_length=input_shape[1])) 
    model.add(Bidirectional(GRU(256, return_sequences = False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(256, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = 'sparse_categorical_crossentropy', 
                 optimizer = Adam(learning_rate))
    return model

tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2]))

encodeco_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print('Original sentence:')
print(logits_to_text_en(tmp_x[:1].flatten(), english_tokenizer))
print('Translated sentence:')
print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/20
110288/110288 [==============================] - 173s 2ms/step - loss: 2.4363 - val_loss: 1.6915
Epoch 2/20
110288/110288 [==============================] - 175s 2ms/step - loss: 1.3726 - val_loss: 1.1235
Epoch 3/20
110288/110288 [==============================] - 177s 2ms/step - loss: 0.9688 - val_loss: 0.8357
Epoch 4/20
110288/110288 [==============================] - 167s 2ms/step - loss: 0.7145 - val_loss: 0.6113
Epoch 5/20
110288/110288 [==============================] - 160s 1ms/step - loss: 0.5060 - val_loss: 0.4110
Epoch 6/20
110288/110288 [==============================] - 160s 1ms/step - loss: 0.3338 - val_loss: 0.3191
Epoch 7/20
110288/110288 [==============================] - 161s 1ms/step - loss: 0.2621 - val_loss: 0.2205
Epoch 8/20
110288/110288 [==============================] - 155s 1ms/step - loss: 0.1778 - val_loss: 0.1699
Epoch 9/20
110288/110288 [==============================] - 149s 1ms/step - loss: 0.1

In [116]:
encodeco_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 15, 64)            12800     
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 512)               493056    
_________________________________________________________________
repeat_vector_13 (RepeatVect (None, 21, 512)           0         
_________________________________________________________________
gru_42 (GRU)                 (None, 21, 256)           590592    
_________________________________________________________________
time_distributed_29 (TimeDis (None, 21, 345)           88665     
Total params: 1,185,113
Trainable params: 1,185,113
Non-trainable params: 0
_________________________________________________________________


## Neural machine translation with attention

One of the problems of the previous model is the fact that the model has to memorize the entire sentence before start to translate it. The attention model introduces and additional layer that weight the contribution of the first bidirectional RNN layer's outputs to be feed into the last recurrent layer.


Dzmitry Bahdanau, Kyunghyun Cho, Yoshua Bengio , Neural Machine Translation by Jointly Learning to Align and Translate. https://arxiv.org/abs/1409.0473

<table>
<td> 
<img src="Images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="Images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **Figure 1**: Neural machine translation with attention</center></caption>

In the following you can see the implementation of the attention model. Please pay attention to:

**1) `one_step_attention()`**: At step $t$, given all the hidden states of the Bi-LSTM ($[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$) and the previous hidden state of the second LSTM ($s^{<t-1>}$), `one_step_attention()` will compute the attention weights ($[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$) and output the context vector (see Figure  1 (right) for details):
$$context^{<t>} = \sum_{t' = 0}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 

Note that we are denoting the attention in this notebook $context^{\langle t \rangle}$. The coefficients must satisfy the constrain $\sum_{t' = 0}^{T_x} \alpha^{<t,t'>} = 1$.
  
**2) `model()`**: Implements the entire model. It first runs the input through a Bi-LSTM to get back $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$. Then, it calls `one_step_attention()` $T_y$ times (`for` loop). At each iteration of this loop, it gives the computed context vector $c^{<t>}$ to the second LSTM, and runs the output of the LSTM through a dense layer with softmax activation to generate a prediction $\hat{y}^{<t>}$. 

In [117]:
# Defined shared layers as global variables
repeator = RepeatVector(max_english_sequence_length)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

In [118]:
n_a = 256
n_s = 256
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(french_vocab_size, activation='softmax')

def attention_model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    e = Embedding(human_vocab_size, 64)(X)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([X, s0, c0], outputs)
    
    ### END CODE HERE ###
    
    return model

In [119]:
model = attention_model(max_english_sequence_length, max_french_sequence_length, n_a, n_s, english_vocab_size, french_vocab_size)
opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)

In [120]:
m = preproc_french_sentences.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(preproc_french_sentences.swapaxes(0,1))

In [121]:
model.fit([tmp_x, s0, c0], outputs, epochs=20, batch_size=1024)
print('Original sentence:')
print(logits_to_text_en(tmp_x[:1].flatten(), english_tokenizer))
print('Translated sentence:')
prediction = model.predict([tmp_x[:1], s0, c0])
output = [logits_to_text(prediction[int(i)], french_tokenizer) for i in range(len(prediction))]
print(' '.join(output))

Epoch 1/20
137860/137860 [==============================] - 606s 4ms/step - loss: 47.7140 - dense_42_loss: 0.3419
Epoch 2/20
137860/137860 [==============================] - 390s 3ms/step - loss: 28.1594 - dense_42_loss: 0.0076
Epoch 3/20
137860/137860 [==============================] - 365s 3ms/step - loss: 21.6951 - dense_42_loss: 0.0035
Epoch 4/20
137860/137860 [==============================] - 374s 3ms/step - loss: 17.6323 - dense_42_loss: 0.0023
Epoch 5/20
137860/137860 [==============================] - 370s 3ms/step - loss: 15.1050 - dense_42_loss: 0.0019
Epoch 6/20
137860/137860 [==============================] - 365s 3ms/step - loss: 13.3470 - dense_42_loss: 0.0014
Epoch 7/20
137860/137860 [==============================] - 343s 2ms/step - loss: 11.9639 - dense_42_loss: 0.0011
Epoch 8/20
137860/137860 [==============================] - 341s 2ms/step - loss: 10.8752 - dense_42_loss: 9.0128e-04
Epoch 9/20
137860/137860 [==============================] - 347s 3ms/step - loss: 9.

In [122]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_32 (Embedding)        (None, 15, 64)       12800       input_16[0][0]                   
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 256)          0                                            
__________________________________________________________________________________________________
bidirectional_29 (Bidirectional (None, 15, 512)      657408      embedding_32[0][0]               
__________________________________________________________________________________________________
repeat_vec

Total params: 1,547,098
Trainable params: 1,547,098
Non-trainable params: 0
__________________________________________________________________________________________________


## Metrics in a real context

From: wikipedia

**BLEU (bilingual evaluation understudy)** is an algorithm for evaluating the quality of text which has been machine-translated from one natural language to another. Quality is considered to be the correspondence between a machine's output and that of a human: "the closer a machine translation is to a professional human translation, the better it is" – this is the central idea behind BLEU. BLEU was one of the first metrics to claim a high correlation with human judgements of quality, and remains one of the most popular automated and inexpensive metrics.

Scores are calculated for individual translated segments—generally sentences—by comparing them with a set of good quality reference translations. Those scores are then averaged over the whole corpus to reach an estimate of the translation's overall quality. Intelligibility or grammatical correctness are not taken into account

NLTK provides the sentence_bleu() function for evaluating a candidate sentence against one or more reference sentences.